In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/:/scratch/wmql/tract_querier/tract_querier/nipype/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [6]:
inputLabelMap = '/Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/JointFusion/JointFusion_HDAtlas20_2015_fs_standard_label.nii.gz'
Baseline_ukfTracts = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/RunUKFt/mapflow/_RunUKFt0/Baseline_ukfTracts.vtp'
NN_ukfTracts = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/RunUKFt/mapflow/_RunUKFt1/NN_ukfTracts.vtp'
IFFT_ukfTracts = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/RunUKFt/mapflow/_RunUKFt2/IFFT_ukfTracts.vtp'
TV_ukfTracts = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/RunUKFt/mapflow/_RunUKFt3/TV_ukfTracts.vtp'
WTV_ukfTracts = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/TractographyWorkflow_CACHE_105115/RunUKFt/mapflow/_RunUKFt4/WTV_ukfTracts.vtp'

In [7]:
sessionID = '105115'
WFname = 'HCPWorkflow_Tractography_' + sessionID

In [11]:
###### UTILITY FUNCTIONS #######
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/#
def MakeInputSRList(Baseline_ukfTracts,NN_ukfTracts,IFFT_ukfTracts,TV_ukfTracts,WTV_ukfTracts):
    imagesList = [Baseline_ukfTracts,NN_ukfTracts,IFFT_ukfTracts,TV_ukfTracts,WTV_ukfTracts]
    return imagesList

def runWMQL(input_tractography, input_atlas):
    import os
    from wmql import TractQuerier
    # prepare proper prefix
    sr_file_name = os.path.basename(input_tractography)
    sr_file_name_base = os.path.splitext(sr_file_name)[0]
    sr_name = sr_file_name_base.split('_',1)[0]
    out_prefix = sr_name + '_query'
    # run WMQL
    tract_querier = TractQuerier()
    tract_querier.inputs.input_atlas = input_atlas
    tract_querier.inputs.input_tractography = input_tractography
    tract_querier.inputs.out_prefix = out_prefix
    tract_querier.inputs.queries = ['cst.left' ,'cst.right']
    tract_querier.run()
    # check outputs
    output_cst_left_name = out_prefix + '_' + 'cst.left.vtp'
    output_cst_right_name = out_prefix + '_' + 'cst.right.vtp'
    output_cst_left = os.path.join(os.getcwd(), output_cst_left_name)
    output_cst_right = os.path.join(os.getcwd(), output_cst_right_name)
    assert os.path.isfile(output_cst_left), "Output cst tract file is not found: %s" % output_cst_left
    assert os.path.isfile(output_cst_right), "Output cst tract file is not found: %s" % output_cst_right
    return [output_cst_left,output_cst_right]

# This function helps to pick desirable output from the output list
def pickFromList(inlist,item):
    return inlist[item]
    
def MakeInputCSTList(NN_cst, IFFT_cst, TV_cst, WTV_cst):
    outputList = [NN_cst, IFFT_cst, TV_cst, WTV_cst]
    return outputList

def CSTOverlap(gs_cst_left, gs_cst_right, sr_cst_left, sr_cst_right):
    import os
    def ComputeBhattacharyyaCoeficient(baseline_bundle, sr_bundle):
        # read in each fiber bundle
        # gs: gold standard, sr: super-resolution reconstructed
        import vtk
        import numpy as np
        ##
        def ReturnDistributionInEachCoordinate(bundle):
            from scipy import stats
            numPoints = bundle.GetNumberOfPoints()
            points = bundle.GetPoints()
            x_arr = np.array([points.GetPoint(i)[0] for i in xrange(numPoints)])
            y_arr = np.array([points.GetPoint(i)[1] for i in xrange(numPoints)])
            z_arr = np.array([points.GetPoint(i)[2] for i in xrange(numPoints)])
            x = np.linspace(x_arr.min(), x_arr.max(), 100)
            y = np.linspace(y_arr.min(), y_arr.max(), 100)
            z = np.linspace(z_arr.min(), z_arr.max(), 100)
            kde_x = stats.gaussian_kde(x_arr)
            kde_y = stats.gaussian_kde(y_arr)
            kde_z = stats.gaussian_kde(z_arr)
            p_x = kde_x(x)
            p_y = kde_y(y)
            p_z = kde_z(z)
            return p_x, p_y, p_z
        ## read in each fiber bundle
        reader_gs = vtk.vtkXMLPolyDataReader()
        reader_gs.SetFileName(baseline_bundle)
        reader_gs.Update()
        gs = reader_gs.GetOutput()
        #
        reader_sr = vtk.vtkXMLPolyDataReader()
        reader_sr.SetFileName(sr_bundle)
        reader_sr.Update()
        sr = reader_sr.GetOutput()
        ## Use ksdensity to estimate probability density function for the sample data in each coordinate
        [p_gs_x, p_gs_y, p_gs_z] = ReturnDistributionInEachCoordinate(gs)
        [p_sr_x, p_sr_y, p_sr_z] = ReturnDistributionInEachCoordinate(sr)
        BC = (1.0/3.0)*( np.sum(np.sqrt(p_gs_x * p_sr_x)) + np.sum(np.sqrt(p_gs_y * p_sr_y)) + np.sum(np.sqrt(p_gs_z * p_sr_z)) )
        return BC
        ##
    def writeLabelStatistics(filename,statsList):
        import csv
        label = os.path.splitext(os.path.basename(filename))[0]
        with open(filename, 'wb') as lf:
            headerdata = [['#Label', 'BC_left.cst', 'BC_right.cst', 'BC_cst']]
            wr = csv.writer(lf, delimiter=',')
            wr.writerows(headerdata)
            wr.writerows([[label] + statsList])
        ###
    # compute Bhattacharyya Coeficient for cst.left/right/total
    bc_l = ComputeBhattacharyyaCoeficient(gs_cst_left,sr_cst_left)
    if bc_l > 1: bc_l = 1
    bc_r = ComputeBhattacharyyaCoeficient(gs_cst_right,sr_cst_right)
    if bc_r > 1: bc_r = 1
    bc_total = (bc_l + bc_r)/2.0
    statsList = [format(bc_l,'.4f'), format(bc_r,'.4f'), format(bc_total,'.4f')]
    # create output file name
    srfn = os.path.basename(sr_cst_left)
    srfnbase = os.path.splitext(srfn)[0]
    srLabel = srfnbase.split('_',1)[0]
    fn = srLabel + '.csv'
    output_csv_file = os.path.join(os.getcwd(), fn)
    # write the stats list
    writeLabelStatistics(output_csv_file,statsList)
    assert os.path.isfile(output_csv_file), "Output Bhattacharyya coeficient file is not found: %s" % output_csv_file
    return output_csv_file
#################################
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\#

TractWF = pe.Workflow(name=WFname)

BASE_DIR = os.path.join('/scratch/TESTS/IpythonNotebook/20160615_HCPWF', '4_tractographyWF')
TractWF.base_dir = BASE_DIR

inputsSpec = pe.Node(interface=IdentityInterface(fields=['inputLabelMap',
                                                         'Baseline_ukfTracts',
                                                          'NN_ukfTracts',
                                                          'IFFT_ukfTracts',
                                                          'TV_ukfTracts',
                                                          'WTV_ukfTracts'
                                                         ]),
                      name='inputsSpec')

inputsSpec.inputs.inputLabelMap = inputLabelMap
inputsSpec.inputs.Baseline_ukfTracts = Baseline_ukfTracts
inputsSpec.inputs.NN_ukfTracts = NN_ukfTracts
inputsSpec.inputs.IFFT_ukfTracts = IFFT_ukfTracts
inputsSpec.inputs.TV_ukfTracts = TV_ukfTracts
inputsSpec.inputs.WTV_ukfTracts = WTV_ukfTracts

outputsSpec = pe.Node(interface=IdentityInterface(fields=['Baseline_cst_left','Baseline_cst_right',
                                                          'NN_cst_left','NN_cst_right',
                                                          'IFFT_cst_left','IFFT_cst_right',
                                                          'TV_cst_left','TV_cst_right',
                                                          'WTV_cst_left','WTV_cst_right',
                                                          'NN_overlap_coeficient',
                                                          'IFFT_overlap_coeficient',
                                                          'TV_overlap_coeficient',
                                                          'WTV_overlap_coeficient'
                                                         ]),
                      name='outputsSpec')

##
## Step 1: Create input list
##
MakeInputSRListNode = pe.Node(Function(function=MakeInputSRList,
                                       input_names=['Baseline_ukfTracts','NN_ukfTracts','IFFT_ukfTracts','TV_ukfTracts','WTV_ukfTracts'],
                                       output_names=['imagesList']),
                              name="MakeInputSRList")
TractWF.connect([(inputsSpec,MakeInputSRListNode,[('Baseline_ukfTracts','Baseline_ukfTracts'),
                                                  ('NN_ukfTracts','NN_ukfTracts'),
                                                  ('IFFT_ukfTracts','IFFT_ukfTracts'),
                                                  ('TV_ukfTracts','TV_ukfTracts'),
                                                  ('WTV_ukfTracts','WTV_ukfTracts')
                                                 ])])

##
## Step 3: run WMQL to extract cortico-spinal (CST) tract bundles
##
tract_querier = pe.MapNode(interface=Function(function = runWMQL,
                                              input_names=['input_tractography','input_atlas'],
                                              output_names=['output_cst_left','output_cst_right']),
                            name="tract_querier", iterfield=['input_tractography'])
TractWF.connect(MakeInputSRListNode,'imagesList',tract_querier,'input_tractography')
TractWF.connect(inputsSpec,'inputLabelMap',tract_querier,'input_atlas')
# baseline cst
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 0),outputsSpec,'Baseline_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 0),outputsSpec,'Baseline_cst_right')
# NN cst
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 1),outputsSpec,'NN_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 1),outputsSpec,'NN_cst_right')
# IFFT cst
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 2),outputsSpec,'IFFT_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 2),outputsSpec,'IFFT_cst_right')
# TV cst
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 3),outputsSpec,'TV_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 3),outputsSpec,'TV_cst_right')
# WTV cst
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 4),outputsSpec,'WTV_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 4),outputsSpec,'WTV_cst_right')

##
## Step 4: Make SR CST left/right lists
##

# step 4_1: input cst.left
cst_left_list = pe.Node(Function(function=MakeInputCSTList,
                                 input_names=['NN_cst', 'IFFT_cst', 'TV_cst', 'WTV_cst'],
                                 output_names=['outputList']),
                        name="cst_left_list")
TractWF.connect([(tract_querier,cst_left_list,[(('output_cst_left', pickFromList, 1),'NN_cst'),
                                               (('output_cst_left', pickFromList, 2),'IFFT_cst'),
                                               (('output_cst_left', pickFromList, 3),'TV_cst'),
                                               (('output_cst_left', pickFromList, 4),'WTV_cst')
                                              ])])

# step 4_2: input cst.right
cst_right_list = pe.Node(Function(function=MakeInputCSTList,
                                  input_names=['NN_cst', 'IFFT_cst', 'TV_cst', 'WTV_cst'],
                                  output_names=['outputList']),
                        name="cst_right_list")
TractWF.connect([(tract_querier,cst_right_list,[(('output_cst_right', pickFromList, 1),'NN_cst'),
                                                (('output_cst_right', pickFromList, 2),'IFFT_cst'),
                                                (('output_cst_right', pickFromList, 3),'TV_cst'),
                                                (('output_cst_right', pickFromList, 4),'WTV_cst')
                                              ])])

#
# Step 5: Compute Bhattacharyya coeficient to find overlap between CST tract bundles
#
cst_overlap = pe.MapNode(interface=Function(function = CSTOverlap,
                                            input_names=['gs_cst_left', 'gs_cst_right', 'sr_cst_left', 'sr_cst_right'],
                                            output_names=['output_csv_file']),
                         name="BhattacharyyaCoeficient", iterfield=['sr_cst_left','sr_cst_right'])
TractWF.connect(tract_querier,('output_cst_left', pickFromList, 0),cst_overlap,'gs_cst_left')
TractWF.connect(tract_querier,('output_cst_right', pickFromList, 0),cst_overlap,'gs_cst_right')
TractWF.connect(cst_left_list,'outputList',cst_overlap,'sr_cst_left')
TractWF.connect(cst_right_list,'outputList',cst_overlap,'sr_cst_right')
TractWF.connect(cst_overlap, ('output_csv_file', pickFromList, 0), outputsSpec, 'NN_overlap_coeficient')
TractWF.connect(cst_overlap, ('output_csv_file', pickFromList, 1), outputsSpec, 'IFFT_overlap_coeficient')
TractWF.connect(cst_overlap, ('output_csv_file', pickFromList, 2), outputsSpec, 'TV_overlap_coeficient')
TractWF.connect(cst_overlap, ('output_csv_file', pickFromList, 3), outputsSpec, 'WTV_overlap_coeficient')

######## DATA Sink
DWIDataSink = pe.Node(interface=nio.DataSink(), name='DWIDataSink')
DWIDataSink.overwrite = True
DWIDataSink.inputs.base_directory = BASE_DIR
DWIDataSink.inputs.substitutions = [('Outputs/Tractography/_RunUKFt0/','Outputs/Tractography/'),
                                    ('Outputs/Tractography/_RunUKFt1/','Outputs/Tractography/'),
                                    ('Outputs/Tractography/_RunUKFt2/','Outputs/Tractography/'),
                                    ('Outputs/Tractography/_RunUKFt3/','Outputs/Tractography/'),
                                    ('Outputs/Tractography/_RunUKFt4/','Outputs/Tractography/'),
                                    ('Outputs/WMQL/_tract_querier0/','Outputs/WMQL/'),
                                    ('Outputs/WMQL/_tract_querier1/','Outputs/WMQL/'),
                                    ('Outputs/WMQL/_tract_querier2/','Outputs/WMQL/'),
                                    ('Outputs/WMQL/_tract_querier3/','Outputs/WMQL/'),
                                    ('Outputs/WMQL/_tract_querier4/','Outputs/WMQL/'),
                                    ('Outputs/Stats/_BhattacharyyaCoeficient0/','Outputs/Stats/'),
                                    ('Outputs/Stats/_BhattacharyyaCoeficient1/','Outputs/Stats/'),
                                    ('Outputs/Stats/_BhattacharyyaCoeficient2/','Outputs/Stats/'),
                                    ('Outputs/Stats/_BhattacharyyaCoeficient3/','Outputs/Stats/')
                                   ]

TractWF.connect(inputsSpec, 'Baseline_ukfTracts', DWIDataSink, 'Outputs.Tractography.@Baseline_ukfTracts')
TractWF.connect(inputsSpec, 'NN_ukfTracts', DWIDataSink, 'Outputs.Tractography.@NN_ukfTracts')
TractWF.connect(inputsSpec, 'IFFT_ukfTracts', DWIDataSink, 'Outputs.Tractography.@IFFT_ukfTracts')
TractWF.connect(inputsSpec, 'TV_ukfTracts', DWIDataSink, 'Outputs.Tractography.@TV_ukfTracts')
TractWF.connect(inputsSpec, 'WTV_ukfTracts', DWIDataSink, 'Outputs.Tractography.@WTV_ukfTracts')
TractWF.connect(outputsSpec, 'Baseline_cst_left', DWIDataSink, 'Outputs.WMQL.@Baseline_cst_left')
TractWF.connect(outputsSpec, 'Baseline_cst_right', DWIDataSink, 'Outputs.WMQL.@Baseline_cst_right')
TractWF.connect(outputsSpec, 'NN_cst_left', DWIDataSink, 'Outputs.WMQL.@NN_cst_left')
TractWF.connect(outputsSpec, 'NN_cst_right', DWIDataSink, 'Outputs.WMQL.@NN_cst_right')
TractWF.connect(outputsSpec, 'IFFT_cst_left', DWIDataSink, 'Outputs.WMQL.@IFFT_cst_left')
TractWF.connect(outputsSpec, 'IFFT_cst_right', DWIDataSink, 'Outputs.WMQL.@IFFT_cst_right')
TractWF.connect(outputsSpec, 'TV_cst_left', DWIDataSink, 'Outputs.WMQL.@TV_cst_left')
TractWF.connect(outputsSpec, 'TV_cst_right', DWIDataSink, 'Outputs.WMQL.@TV_cst_right')
TractWF.connect(outputsSpec, 'WTV_cst_left', DWIDataSink, 'Outputs.WMQL.@WTV_cst_left')
TractWF.connect(outputsSpec, 'WTV_cst_right', DWIDataSink, 'Outputs.WMQL.@WTV_cst_right')
TractWF.connect(outputsSpec, 'NN_overlap_coeficient', DWIDataSink, 'Outputs.Stats.@NN_overlap_coeficient')
TractWF.connect(outputsSpec, 'IFFT_overlap_coeficient', DWIDataSink, 'Outputs.Stats.@IFFT_overlap_coeficient')
TractWF.connect(outputsSpec, 'TV_overlap_coeficient', DWIDataSink, 'Outputs.Stats.@TV_overlap_coeficient')
TractWF.connect(outputsSpec, 'WTV_overlap_coeficient', DWIDataSink, 'Outputs.Stats.@WTV_overlap_coeficient')
#

TractWF.write_graph()
TractWF.run()